# Programming Agent Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a MemGPT client 

In [2]:
from letta import create_client 

client = create_client() 

/Users/azinasgarian/miniconda3/envs/mem/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Letta.letta.server.server - INFO - Creating sqlite engine sqlite:////Users/azinasgarian/.letta/sqlite.db


In [3]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig


# Set the default llm config
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

# Set the default embedding config
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

## Section 1: Memory Blocks 

### Understanding ChatMemory

In [4]:
from letta.schemas.memory import ChatMemory  

In [5]:
chat_memory = ChatMemory(
    human="Name: Bob", 
    persona="You are a helpful assistant"
)

In [6]:
chat_memory.list_block_labels()

['persona', 'human']

In [7]:
chat_memory.get_block("human")

Block(value='Name: Bob', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-eed4729c-085e-4006-a92c-0e80b5206336', organization_id=None, created_by_id=None, last_updated_by_id=None)

In [8]:
import inspect

In [9]:
print(inspect.getsource(chat_memory.core_memory_append))

    def core_memory_append(agent_state: "AgentState", label: str, content: str) -> Optional[str]:  # type: ignore
        """
        Append to the contents of core memory.

        Args:
            label (str): Section of the memory to be edited (persona or human).
            content (str): Content to write to the memory. All unicode (including emojis) are supported.

        Returns:
            Optional[str]: None is always returned as this function does not produce a response.
        """
        current_value = str(agent_state.memory.get_block(label).value)
        new_value = current_value + "\n" + str(content)
        agent_state.memory.update_block_value(label=label, value=new_value)
        return None



#### Context compilation 

In [10]:
chat_memory.get_prompt_template()

'{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

In [11]:
chat_memory.compile()

'<persona characters="27/5000">\nYou are a helpful assistant\n</persona>\n<human characters="9/5000">\nName: Bob\n</human>'

## Section 2: Defining a custom memory module

In this section, we're going to define a custom memory module called `TaskMemory`. This module will be used to store and manage tasks for a user in the core memory along with the human and persona. We'll also implement 2 functions to manage the task queue called `task_queue_push` and `task_queue_pop`.

### Defining a memory module


In [12]:
import json
from typing import Optional, List

from letta.schemas.block import Block
from letta.schemas.memory import ChatMemory

In [13]:
class TaskMemory(ChatMemory):
    def __init__(self, human: str, persona: str, tasks: Optional[List[str]] = None):
        """
        Initialize TaskMemory with a list of tasks.

        Args:
            human (str): The human identifier.
            persona (str): The persona identifier.
            tasks (Optional[List[str]]): An initial list of tasks. Defaults to an empty list.
        """
        if tasks is None:
            tasks = []
        super().__init__(human=human, persona=persona, limit=2000)
        # Store the tasks as a JSON string in a memory block labeled "tasks"
        self.set_block(Block(limit=2000, value=json.dumps(tasks), label="tasks"))

    def task_queue_push(self: "Agent", task_description: str):
        """
        Push to a task queue stored in core memory. 

        Args:
            task_description (str): A description of the next task you must accomplish. 
            
        Returns:
            Optional[str]: None is always returned as this function 
            does not produce a response.
        """
        tasks = json.loads(self.memory.get_block("tasks").value)
        tasks.append(task_description)
        self.memory.update_block_value("tasks", json.dumps(tasks))
        return None

    def task_queue_pop(self: "Agent"):
        """
        Get the next task from the task queue 
 
        Returns:
            Optional[str]: The description of the task popped from the 
            queue, if there are still tasks in queue. Otherwise, returns
            None (the task queue is empty)
        """
        tasks = json.loads(self.get_block("tasks").value)
        if not tasks:
            return None
        # Remove and return the first task in the list
        task = tasks.pop(0)
        self.update_block_value("tasks", json.dumps(tasks))
        return task

### Creating an agent with custom `TaskMemory`

In [14]:
task_agent_name = "task_agent"

task_agent_state = client.create_agent(
    name=task_agent_name, 
    system = open("task_queue_system_prompt.txt", "r").read(),
    memory=TaskMemory(
        human="My name is Azin", 
        persona="You are an agent that must clear its tasks.", 
        tasks=None
    )
)

In [15]:
client.get_core_memory(task_agent_state.id).get_blocks()

[Block(value='[]', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-0c9891c6-2918-4f61-ad13-1990e5acf202', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-32d3705c-8787-4b0c-a142-ee385a4a7f22', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-ebf3435f-4d08-4ccd-b309-6022f1780f4c', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [16]:
message = "Add these 2 tasks: 1. Explain my name, Task 2. Tell me a story about my name in less than 100 words"

In [17]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message=message
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-37fd8c10-da3b-46b2-9467-d86ced8eba68 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 11, 14, 26, 32, 101586, tzinfo=datetime.timezone.utc) updated_at=None id='message-fa44520d-fe18-4228-840e-644f827ecd47' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User requested two tasks related to their name, preparing to add them to the task list.')] organization_id=None agent_id='agent-37fd8c10-da3b-46b2-9467-d86ced8eba68' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_4YLPF76Wd89gDN8OsGkXSDsJ', function=Function(arguments='{\n  "label": "tasks",\n  "content": "1. Explain my name 2. Tell me a story about my name in less than 100 words",\n  "request_heartbeat": true\n}', name='core_memory_append'), type='function')] tool_call_id=Non

LettaResponse(messages=[ReasoningMessage(id='message-fa44520d-fe18-4228-840e-644f827ecd47', date=datetime.datetime(2025, 2, 11, 14, 26, 32, 101586, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User requested two tasks related to their name, preparing to add them to the task list.'), ToolCallMessage(id='message-fa44520d-fe18-4228-840e-644f827ecd47', date=datetime.datetime(2025, 2, 11, 14, 26, 32, 101586, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_append', arguments='{\n  "label": "tasks",\n  "content": "1. Explain my name 2. Tell me a story about my name in less than 100 words",\n  "request_heartbeat": true\n}', tool_call_id='call_4YLPF76Wd89gDN8OsGkXSDsJ')), ToolReturnMessage(id='message-049c3e84-3a44-4d57-b26c-5a23173f3772', date=datetime.datetime(2025, 2, 11, 14, 26, 32, 196368, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:26:32 AM EST-0500"\n}', status='success', tool_call_id='call_4YLPF76Wd89gDN8OsGkXSDsJ', stdout=None, stderr=None), ReasoningMessage(id='message-a46052d6-400a-491e-ab63-15d2ff520fe3', date=datetime.datetime(2025, 2, 11, 14, 26, 36, 111858, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Tasks successfully added. Now ready to explain and tell a story about the user's name."), ToolCallMessage(id='message-a46052d6-400a-491e-ab63-15d2ff520fe3', date=datetime.datetime(2025, 2, 11, 14, 26, 36, 111858, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "I\'ve added the tasks: explaining your name and telling a story about it!"\n}', tool_call_id='call_xN0PcBhSAaJGgepEgwxwhiXI')), ToolReturnMessage(id='message-b542fe69-8db2-4d50-b1c3-dfae9973b1fd', date=datetime.datetime(2025, 2, 11, 14, 26, 36, 118101, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:26:36 AM EST-0500"\n}', status='success', tool_call_id='call_xN0PcBhSAaJGgepEgwxwhiXI', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=120, prompt_tokens=4699, total_tokens=4819, step_count=2))

In [18]:
# task_agent_state.memory.get_blocks() # accesses the memory from the agent's in-memory state, which might not be synchronized with the database if changes haven't been persisted.
client.get_core_memory(task_agent_state.id).get_blocks() # accesses the memory directly from the database/storage layer through the client interface. This shows the actual persisted state.

[Block(value='[]\n1. Explain my name 2. Tell me a story about my name in less than 100 words', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-0c9891c6-2918-4f61-ad13-1990e5acf202', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-32d3705c-8787-4b0c-a142-ee385a4a7f22', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-ebf3435f-4d08-4ccd-b309-6022f1780f4c', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [19]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-37fd8c10-da3b-46b2-9467-d86ced8eba68 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 11, 14, 27, 40, 919025, tzinfo=datetime.timezone.utc) updated_at=None id='message-90e94ab1-4ea4-4ff5-bb5a-0b4416a2bc0d' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User requested to complete the tasks. Starting with the explanation of their name.')] organization_id=None agent_id='agent-37fd8c10-da3b-46b2-9467-d86ced8eba68' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_qJ3b1FKX4oPha0uonha6UYqJ', function=Function(arguments='{\n  "label": "tasks",\n  "old_content": "1. Explain my name 2. Tell me a story about my name in less than 100 words",\n  "new_content": "2. Tell me a story about my name in less than 100 words",\n  "request_heartbea

LettaResponse(messages=[ReasoningMessage(id='message-90e94ab1-4ea4-4ff5-bb5a-0b4416a2bc0d', date=datetime.datetime(2025, 2, 11, 14, 27, 40, 919025, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User requested to complete the tasks. Starting with the explanation of their name.'), ToolCallMessage(id='message-90e94ab1-4ea4-4ff5-bb5a-0b4416a2bc0d', date=datetime.datetime(2025, 2, 11, 14, 27, 40, 919025, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "tasks",\n  "old_content": "1. Explain my name 2. Tell me a story about my name in less than 100 words",\n  "new_content": "2. Tell me a story about my name in less than 100 words",\n  "request_heartbeat": true\n}', tool_call_id='call_qJ3b1FKX4oPha0uonha6UYqJ')), ToolReturnMessage(id='message-8c1eb5ea-e7e9-4fbd-b3fc-798befcf4030', date=datetime.datetime(2025, 2, 11, 14, 27, 41, 26379, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:27:41 AM EST-0500"\n}', status='success', tool_call_id='call_qJ3b1FKX4oPha0uonha6UYqJ', stdout=None, stderr=None), ReasoningMessage(id='message-91c9affa-713c-48b4-b0e2-c8c03d83475e', date=datetime.datetime(2025, 2, 11, 14, 27, 42, 980001, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Now to explain the user's name, Azin, before moving to the story task."), ToolCallMessage(id='message-91c9affa-713c-48b4-b0e2-c8c03d83475e', date=datetime.datetime(2025, 2, 11, 14, 27, 42, 980001, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Your name, Azin, is of Persian origin, meaning \\"brave\\" or \\"courageous.\\" It carries a sense of strength and valor."\n}', tool_call_id='call_4RDSSdwrbJa3fErhxNuQnqo1')), ToolReturnMessage(id='message-4feac591-9c54-4840-8a6f-75103449541a', date=datetime.datetime(2025, 2, 11, 14, 27, 42, 985733, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:27:42 AM EST-0500"\n}', status='success', tool_call_id='call_4RDSSdwrbJa3fErhxNuQnqo1', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=153, prompt_tokens=5376, total_tokens=5529, step_count=2))

In [20]:
client.get_core_memory(task_agent_state.id).get_blocks()

[Block(value='[]\n2. Tell me a story about my name in less than 100 words', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-0c9891c6-2918-4f61-ad13-1990e5acf202', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-32d3705c-8787-4b0c-a142-ee385a4a7f22', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-ebf3435f-4d08-4ccd-b309-6022f1780f4c', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [21]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-37fd8c10-da3b-46b2-9467-d86ced8eba68 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 11, 14, 28, 49, 304521, tzinfo=datetime.timezone.utc) updated_at=None id='message-8ceab90d-4e62-4523-ab1f-6abe80c516d0' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="Completing the final task of telling a story about the user's name.")] organization_id=None agent_id='agent-37fd8c10-da3b-46b2-9467-d86ced8eba68' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_bBnW7pK7YopfBUKNkXTA1avs', function=Function(arguments='{\n  "label": "tasks",\n  "old_content": "2. Tell me a story about my name in less than 100 words",\n  "new_content": "",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_id=No

LettaResponse(messages=[ReasoningMessage(id='message-8ceab90d-4e62-4523-ab1f-6abe80c516d0', date=datetime.datetime(2025, 2, 11, 14, 28, 49, 304521, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Completing the final task of telling a story about the user's name."), ToolCallMessage(id='message-8ceab90d-4e62-4523-ab1f-6abe80c516d0', date=datetime.datetime(2025, 2, 11, 14, 28, 49, 304521, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "tasks",\n  "old_content": "2. Tell me a story about my name in less than 100 words",\n  "new_content": "",\n  "request_heartbeat": true\n}', tool_call_id='call_bBnW7pK7YopfBUKNkXTA1avs')), ToolReturnMessage(id='message-5bce655d-bccd-4c23-8e57-7355d962bbcb', date=datetime.datetime(2025, 2, 11, 14, 28, 49, 419858, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:28:49 AM EST-0500"\n}', status='success', tool_call_id='call_bBnW7pK7YopfBUKNkXTA1avs', stdout=None, stderr=None), ReasoningMessage(id='message-4e65091f-e4a0-4fe4-a1a6-08f87eea36f4', date=datetime.datetime(2025, 2, 11, 14, 28, 51, 777347, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Crafting a short story about the user's name, Azin."), ToolCallMessage(id='message-4e65091f-e4a0-4fe4-a1a6-08f87eea36f4', date=datetime.datetime(2025, 2, 11, 14, 28, 51, 777347, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Once upon a time, in a land of warriors, there lived a brave soul named Azin. With unwavering courage, Azin stood against fears, inspiring others to find their strength. Legends spoke of Azin\'s valor, reminding all that true bravery lies within."\n}', tool_call_id='call_pb6lUA2jLD8ls0d1mwnxw5jy')), ToolReturnMessage(id='message-81581a1c-fb65-4523-9059-602273ae8031', date=datetime.datetime(2025, 2, 11, 14, 28, 51, 781012, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:28:51 AM EST-0500"\n}', status='success', tool_call_id='call_pb6lUA2jLD8ls0d1mwnxw5jy', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=148, prompt_tokens=6041, total_tokens=6189, step_count=2))

> copy the id='block-...' string, from the code cell above "client.get_core_memory...", and then paste into the code cell client.get_block('block-...')

In [22]:
client.get_core_memory(task_agent_state.id).get_blocks()

[Block(value='[]\n', limit=2000, template_name=None, is_template=False, label='tasks', description=None, metadata={}, id='block-0c9891c6-2918-4f61-ad13-1990e5acf202', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Human(value='My name is Azin', limit=2000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-32d3705c-8787-4b0c-a142-ee385a4a7f22', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None),
 Persona(value='You are an agent that must clear its tasks.', limit=2000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-ebf3435f-4d08-4ccd-b309-6022f1780f4c', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)]

In [23]:
print(client.get_block('cut_and_paste_id_from_above'))

None


In [24]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-37fd8c10-da3b-46b2-9467-d86ced8eba68 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 11, 14, 29, 11, 821304, tzinfo=datetime.timezone.utc) updated_at=None id='message-c4c12f9a-da34-4da9-97c2-31f1cb11ab3f' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User seems to want confirmation of task completion. I should clarify that all tasks are done.')] organization_id=None agent_id='agent-37fd8c10-da3b-46b2-9467-d86ced8eba68' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_BOd8ABChDU8v4CXwcUDV7DNc', function=Function(arguments='{\n  "message": "All tasks related to your name are complete! I explained the meaning of your name and shared a short story about it. Let me know if there\'s anything else you\'d like!"\n}', name='send_mes

LettaResponse(messages=[ReasoningMessage(id='message-c4c12f9a-da34-4da9-97c2-31f1cb11ab3f', date=datetime.datetime(2025, 2, 11, 14, 29, 11, 821304, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User seems to want confirmation of task completion. I should clarify that all tasks are done.'), ToolCallMessage(id='message-c4c12f9a-da34-4da9-97c2-31f1cb11ab3f', date=datetime.datetime(2025, 2, 11, 14, 29, 11, 821304, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "All tasks related to your name are complete! I explained the meaning of your name and shared a short story about it. Let me know if there\'s anything else you\'d like!"\n}', tool_call_id='call_BOd8ABChDU8v4CXwcUDV7DNc')), ToolReturnMessage(id='message-29cbc49d-a3db-4df1-bba4-c73a14ddd49f', date=datetime.datetime(2025, 2, 11, 14, 29, 11, 828029, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-11 09:29:11 AM EST-0500"\n}', status='success', tool_call_id='call_BOd8ABChDU8v4CXwcUDV7DNc', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=71, prompt_tokens=3276, total_tokens=3347, step_count=1))